In [1]:
import logfire
from loguru import logger

logger.configure(handlers=[logfire.loguru_handler()])

[1]

In [2]:
from rich import print

from agents.tool_executor import ToolRegistry
from agents.llm import create_tool_use_llm
from agents.experimental.hermes.functions import wikipedia

In [3]:
llm = create_tool_use_llm(verbose=False, n_gpu_layers=-1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
registry = ToolRegistry()
registry.register_tool(wikipedia)

In [5]:
print(wikipedia)

WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        wiki_client=<module 'wikipedia' from 
'/Users/aniket/miniconda3/envs/am/lib/python3.10/site-packages/wikipedia/__init__.py'>,
        top_k_results=1,
        lang='en',
        load_all_available_meta=False,
        doc_content_chars_max=4000
    )
)

In [6]:
registry._openai_tools["wikipedia"]["parameters"]["properties"] = wikipedia.args
registry._openai_tools["wikipedia"]["parameters"]["required"] = "query"

In [7]:
print(registry.openai_tools)

[
    {
        'type': 'function',
        'function': {
            'name': 'wikipedia',
            'description': 'A wrapper around Wikipedia. Useful for when you need to answer general questions about 
people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
            'parameters': {
                'properties': {'query': {'title': 'Query', 'type': 'string'}},
                'required': 'query',
                'type': 'object'
            }
        }
    }
]

In [8]:
output = llm.create_chat_completion(
    messages=[{"role": "user", "content": "Who won the mayor election in London in 2024"}],
    tools=registry.openai_tools,
)
tool_response = registry.call_tool(output)
# print(tool_response)

No Logfire project credentials found.
All data sent to Logfire must be associated with a project.



Do you want to use one of your existing projects?  [y/n] (y):

y


Please select one of the following projects by number:
1. aniketmaurya/agents
 [1] (1):

1


Project initialized successfully. You will be able to view it at: https://logfire.pydantic.dev/aniketmaurya/agents
Press Enter to continue:

Logfire project URL: ]8;id=637266;https://logfire.pydantic.dev/aniketmaurya/agents\https://logfire.pydantic.dev/aniketmaurya/agents]8;;\

02:07:36.988 Function wikipedia invoked with {'query': 'mayor of London 2024'}
02:07:53.238 Function wikipedia, responded with Page: 2024 London mayoral election
Summary: The 2024 London mayoral election was held on 2 May 2024 to elect the next mayor of London. It took place simultaneously with elections to the London Assembly, some local council by-elections in London and regular local elections elsewhere in England and Wales. Following the Elections Act 2022, voting in this election took place under the first-past-the-post system for the first time, replacing the supplementary vote system. The results of the election were announced on 4 May 2024.
Sadiq Khan of the Labour Party, who has served as the mayor of London since 2016, was re-elected as mayor and became the first person to be elected to the role for three terms. The Conservative Party stood former London Assembly Conservative leader Susan Hall as their candidate, the Green Party nominated Hackney borough councillor Zoë Garbet